In [ ]:
import pandas as pd
from urllib.parse import urlparse
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
!pip install validators==0.20.0
import validators
import os.path
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split




# Load the Dataset
#df = pd.read_csv('../CSCT Masters Project/Datasets I Used/List of URLs.csv')
data = pd.read_csv('List of URLs.csv')
#data.head()
print(data.shape)
data.type.value_counts()

In [ ]:
#count of missing values in each column

missing = data.isnull().sum()
print(f"Number of missing values: {missing}")


before_duplicates = data.duplicated().sum()
print(f"Number of duplicate rows: {before_duplicates}")
df = data.drop_duplicates()
after_duplicates = df.duplicated().sum()
print(f"Number of duplicate rows after removing duplicates: {after_duplicates}")


In [ ]:
print(df.shape)
df.type.value_counts()

In [ ]:
df['length of url'] = df['url'].apply(lambda i: len(str(i)))

def having_ip_address(url):
  return 1 if validators.ip_address.ipv4(url) or validators.ip_address.ipv6(url) else 0

df['use of ip'] = df['url'].apply(lambda i: having_ip_address(i))
df.head()

In [ ]:
def use_of_hostname(url):
    hostname = urlparse(url).hostname
    if hostname is None:  # Handle the case when hostname is None
        return 0
    return 1 if hostname in url else 0

df['use of hostname'] = df['url'].apply(use_of_hostname)
df['length of hostname'] = df['url'].apply(lambda i: len(urlparse(i).netloc))

df.head()

In [ ]:
def count_of_dir(url):
    urldir = urlparse(url).path
    return urldir.count('/')
df['count of /'] = df['url'].apply(lambda i: no_of_dir(i))

def count_of_SchemeSeparator(url):
    urldir = urlparse(url).path
    return urldir.count('//')
df['count of //'] = df['url'].apply(lambda i: no_of_SchemeSeparator(i))

In [ ]:
# Length of First Directory
def fd_length(url):
    parsed_url = urlparse(url)
    path_parts = parsed_url.path.split('/')
    if len(path_parts) > 1:
        return len(path_parts[1])
    else:
        return 0
df['length of first directory'] = df['url'].apply(fd_length)

# Length of Top Level Domain
def tld_length(url):
    parsed_url = urlparse(url)
    return len(parsed_url.netloc)
df['length of top level domain'] = df['url'].apply(tld_length)

df.head()

In [ ]:
def count_of_subdomains(url):
  parsed_url = urlparse(url)
  # Handling the case where hostname is None
  if parsed_url.hostname is None:
    return 0  # return 0 when a url is missing hostname
  subdomains = parsed_url.hostname.split('.')
  return len(subdomains) - 2

df['count of subdomains'] = df['url'].apply(count_of_subdomains)

df.head()

In [ ]:
# checking whether a url shortening service has been used in the url or not

shorteners = [
    "bit.ly", "goo.gl", "shorte.st", "go2l.ink", "x.co", "ow.ly", "t.co",
    "tinyurl", "tr.im", "is.gd", "cli.gs", "yfrog.com", "migre.me",
    "ff.im", "tiny.cc", "url4.eu", "twit.ac", "su.pr", "twurl.nl",
    "snipurl.com", "short.to", "BudURL.com", "ping.fm", "post.ly",
    "Just.as", "bkite.com", "snipr.com", "fic.kr", "loopt.us",
    "doiop.com", "short.ie", "kl.am", "wp.me", "rubyurl.com", "om.ly",
    "to.ly", "bit.do", "t.co", "lnkd.in", "db.tt", "qr.ae", "adf.ly",
    "goo.gl", "bitly.com", "cur.lv", "tinyurl.com", "ow.ly", "bit.ly",
    "ity.im", "q.gs", "is.gd", "po.st", "bc.vc", "twitthis.com", "u.to",
    "j.mp", "buzurl.com", "cutt.us", "u.bb", "yourls.org", "x.co",
    "prettylinkpro.com", "scrnch.me", "filoops.info", "vzturl.com",
    "qr.net", "1url.com", "tweez.me", "v.gd", "tr.im", "link.zip.net"
]

def shortening_service(url):
    for shortener in shorteners:
        if shortener in url:
            return 1
    return 0

df['use of url shortner'] = df['url'].apply(lambda i: shortening_service(i))
df.head()

In [ ]:
df['count of https'] = df['url'].apply(lambda i : i.count('https'))
df['count of http'] = df['url'].apply(lambda i : i.count('http'))

In [ ]:
# counting multiple special characters
df['count of %'] = df['url'].apply(lambda i: i.count('%'))
df['count of ?'] = df['url'].apply(lambda i: i.count('?'))
df['count of -'] = df['url'].apply(lambda i: i.count('-'))
df['count of ='] = df['url'].apply(lambda i: i.count('='))
df['count of .'] = df['url'].apply(lambda i: i.count('.'))
df['count of www'] = df['url'].apply(lambda i: i.count('www'))
df['count of @'] = df['url'].apply(lambda i: i.count('@'))
df['count of #'] = df['url'].apply(lambda i: i.count('#'))
df['count of !'] = df['url'].apply(lambda i: i.count('!'))
df['count of $'] = df['url'].apply(lambda i: i.count('$'))
df['count of *'] = df['url'].apply(lambda i: i.count('*'))
df['count of _'] = df['url'].apply(lambda i: i.count('_'))

In [ ]:
df['count of numbers'] = df['url'].str.count('\d')

df['count of letters'] = df['url'].str.count('[a-zA-Z]')
df.head()

In [ ]:
lb_make = LabelEncoder()
df["type_code"] = lb_make.fit_transform(df["type"])
print(df["type_code"].value_counts())
df.to_csv('my_data.csv', index=False)

In [ ]:
X = df.drop(['type_code', 'url', 'type'], axis=1)  # Predictor variables - all columns except type_code, url, and type
y = df['type_code']              # Target variable


#print("\nTarget variable (y):")
#print(y.head())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np

# Assuming X and y are already defined

# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Perform cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(rf_classifier, X, y, cv=cv, scoring='accuracy')

# Print cross-validation scores
print("Cross-validation scores:", scores)
print("Average cross-validation score:", np.mean(scores))

# Train the model on the entire dataset
rf_classifier.fit(X, y)

# Make predictions on the test set (assuming you have a separate test set)
# Replace X_test with your actual test data
y_pred = rf_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("\nAccuracy on test set:", accuracy)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

In [ ]:
clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000) # Adjust hyperparameters as needed
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

scores = cross_val_score(clf, X, y, cv=5)
print("Cross-validation scores:", scores)
print("Average cross-validation score:", scores.mean())

cm = confusion_matrix(y_test, y_pred)
print("Confusion matrix:\n", cm)

report = classification_report(y_test, y_pred)
print("Classification report:\n", report)

Accuracy: 0.9515071163969585
Cross-validation scores: [0.95238058 0.97126145 0.95065705 0.92556834 0.81294599]
Average cross-validation score: 0.9225626827841683
Confusion matrix:
 [[84530    65    16   893]
 [  115 18704    60   263]
 [   83   203  4114   307]
 [ 3227   655   331 14659]]
Classification report:
               precision    recall  f1-score   support

           0       0.96      0.99      0.97     85504
           1       0.95      0.98      0.96     19142
           2       0.91      0.87      0.89      4707
           3       0.91      0.78      0.84     18872

    accuracy                           0.95    128225
   macro avg       0.93      0.90      0.92    128225
weighted avg       0.95      0.95      0.95    128225



In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score

# Assuming X_train, X_test, y_train, y_test are already defined

# Initialize the Gaussian Naive Bayes classifier
nb_classifier = GaussianNB()

# Train the model
nb_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = nb_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

scores = cross_val_score(nb_classifier, X, y, cv=5)
print("Cross-validation scores:", scores)
print("Average cross-validation score:", scores.mean())

cm = confusion_matrix(y_test, y_pred)
print("Confusion matrix:\n", cm)

report = classification_report(y_test, y_pred)
print("Classification report:\n", report)

Accuracy: 0.7829830376291674
Cross-validation scores: [0.78309222 0.78032365 0.82179762 0.77971534 0.73140963]
Average cross-validation score: 0.7792676935075062
Confusion matrix:
 [[78845  5198   735   726]
 [   18 19069    28    27]
 [  252  3021  1396    38]
 [13865  2961   958  1088]]
Classification report:
               precision    recall  f1-score   support

           0       0.85      0.92      0.88     85504
           1       0.63      1.00      0.77     19142
           2       0.45      0.30      0.36      4707
           3       0.58      0.06      0.10     18872

    accuracy                           0.78    128225
   macro avg       0.63      0.57      0.53    128225
weighted avg       0.76      0.78      0.73    128225



In [51]:
import random
import string

def random_url():
    domain = ''.join(random.choices(string.ascii_lowercase, k=8)) + ".com"
    path = ''.join(random.choices(string.ascii_lowercase + string.digits, k=10))
    return f"http://{domain}/{path}"

print(random_url())

http://cpwkqhdj.com/7n88g7st7s


In [60]:
#new_url = random_url()
new_url = input("Enter a URL: ")

new_url_features = {
    'length of url': len(new_url),
    'use of ip': having_ip_address(new_url),
    'use of hostname': use_of_hostname(new_url),
    'length of hostname': len(urlparse(new_url).netloc),
    'count of /': count_of_dir(new_url),
    'count of //': count_of_SchemeSeparator(new_url),
    'length of first directory': fd_length(new_url),
    'length of top level domain': tld_length(new_url),
    'count of subdomains': count_of_subdomains(new_url),
    'use of url shortner': shortening_service(new_url),
    'count of https': new_url.count('https'),
    'count of http': new_url.count('http'),
    'count of %': new_url.count('%'),
    'count of ?': new_url.count('?'),
    'count of -': new_url.count('-'),
    'count of =': new_url.count('='),
    'count of .': new_url.count('.'),
    'count of www': new_url.count('www'),
    'count of @': new_url.count('@'),
    'count of #': new_url.count('#'),
    'count of !': new_url.count('!'),
    'count of $': new_url.count('$'),
    'count of *': new_url.count('*'),
    'count of _': new_url.count('_'),
    'count of numbers': new_url.count('\d'),
    'count of letters': new_url.count('[a-zA-Z]')
}

# Create a DataFrame for the new URL
new_url_df = pd.DataFrame([new_url_features])

# RandomForest Prediction
model = RandomForestClassifier()
model.fit(X_train, y_train)
prediction = model.predict(new_url_df)

# Interpret the prediction (0 for benign, 1 for malicious, etc.)
print("Random Forest Prediction:", prediction)

# Naive Bayes prediction
nb_prediction = nb_classifier.predict(new_url_df)
print("Naive Bayes Prediction:", nb_prediction)

# MLPClassifier prediction
mlp_prediction = clf.predict(new_url_df)
print("MLPClassifier Prediction:", mlp_prediction)

Enter a URL: https://ods.od.nih.gov/factsheets/VitaminD-HealthProfessional/
Random Forest Prediction: [3]
Naive Bayes Prediction: [2]
MLPClassifier Prediction: [3]
